In [196]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import time
from dateutil.relativedelta import relativedelta

In [697]:
Par_rates = pd.read_excel('SOFR PAR.xlsx')
calendar = pd.read_excel('INR_Calendar.xlsx')
d_con1 = 'Act/360'
d_con2 = 'Act/365'
settlement_con = 1
payment_lag = 2
Freq = 'Annual'

In [698]:
cal = calendar
cal = [str(i) for i in cal['Working_Days']]

In [699]:
def check_calendar(x):
    x = str(x)
    y = x in cal
    return(y)


In [700]:
def division(days, d_con):
    out = []
    days = np.array(days)
    if(d_con == 'Act/360'): out = days/360
    if(d_con == 'Act/365'): out = days/365
    return(out)

def fra(end_df, start_df, days, d_con):
    out = (start_df/end_df - 1)*(1/division(days, d_con))
    return(out)

def diff_from_start(start, vec, d_con):
    out = []
    for i in range(len(vec)):
        out.append((vec[i] - start).days)
    out = division(out, d_con)
    return(out)

In [701]:
Tenors = Par_rates.columns.values
Tenors

array(['Dates', 'ON', '1W', '2W', '1M', '2M', '3M', '4M', '5M', '6M',
       '7M', '8M', '9M', '10M', '11M', '12M', '2Y', '3Y', '4Y', '5Y',
       '7Y', '10Y'], dtype=object)

In [702]:
def make_tenors(k):
    
    end_date = []
    spot_date = Par_rates.iloc[k-settlement_con, 0]
    
    for i in range(1, len(Tenors)):
        if(Tenors[i][len(Tenors[i])-1] == 'N'):
            t = Par_rates['Dates'][k-3]
            end_date.append(t)
        
        elif(Tenors[i][len(Tenors[i])-1] == 'W'):
            t = spot_date + relativedelta(days = int(Tenors[i][0:(len(Tenors[i])-1)])*7 )
            if(check_calendar(t) == True):
                end_date.append(t)
            while(check_calendar(t) == False):
                t = t + relativedelta(days = 1)
                if(check_calendar(t) == True):
                    end_date.append(t)
                    
        elif(Tenors[i][len(Tenors[i])-1] == 'M'):
            t = spot_date + relativedelta(months = int(Tenors[i][0:(len(Tenors[i])-1)]) )
            if(check_calendar(t) == True):
                end_date.append(t)
            while(check_calendar(t) == False):
                t = t + relativedelta(days = 1)
                if(check_calendar(t) == True):
                    end_date.append(t)             

        elif(Tenors[i][len(Tenors[i])-1] == 'Y'):
            t = spot_date + relativedelta(years = int(Tenors[i][0:(len(Tenors[i])-1)]) )
            if(check_calendar(t) == True):
                end_date.append(t)
            while(check_calendar(t) == False):
                t = t + relativedelta(days = 1)
                if(check_calendar(t) == True):
                    end_date.append(t)  
                    
    return(end_date, spot_date)


In [703]:
def search_buisness_days(k):
    
    start_dates = []
    ending_dates = []
    
    end_date, spot_date = make_tenors(k)
    
    j = len(end_date)
    
    for i in range(0, j):
        if(i==0):
            start_dates.append(Par_rates.iloc[k,0])
            ending_dates.append(Par_rates.iloc[k-3,0])
            
        else:
            
            s1 = pd.DataFrame(data = list(calendar['Working_Days']), columns = ['List']).reset_index()
            p1 = pd.DataFrame(data = list([end_date[i], spot_date]), columns = ['List'])
            out1 = s1.merge(p1, on = 'List', how = 'right')
            out = out1['index']
            out = list(out)
            out.sort()
            start = calendar['Working_Days'][out[0]: out[1]+1]
            start = list(start)
            start_dates.append(start[0:(len(start)-1)])
            ending_dates.append(start[1:len(start)])
    return(start_dates, ending_dates)
            
            
            
            

In [704]:
def Tenor_end_dates(k):
    
    end_date, spot = make_tenors(k)
    
    to_date = [end_date[0]]
    
    for i in range(1, len(Tenors)-1):
        u = calendar[calendar['Working_Days'] == end_date[i]].index[0]
        to_date.append(calendar['Working_Days'][u+settlement_con])
        
    return(to_date)

In [705]:
def find_payment_dates(end, start, k):
    
    end_date, spot_date = make_tenors(k)
    h = int(pd.Series(end[len(end)-1] - start[0]).dt.days)
    start1 = [spot_date]
    
    i = 0 
    
    while(h>368):
        
        s = start1[0] + relativedelta(years = i + 1)
        if(check_calendar(s) == True):
            start1.append(s)
        else:
            while(check_calendar(s) == False):
                s = s + relativedelta(days = 1)
                if(check_calendar(s) == True):
                    start1.append(s)
        
        h = int(pd.Series(end[len(end)-1] - start1[i]).dt.days)
        i = i+1
        
    return(start1)

            

In [706]:
def float_blocks(start, end, x):
    s1 = pd.DataFrame(start, columns = ['List']).reset_index()
    p1 = pd.DataFrame(x, columns = ['List'])
    out1 = s1.merge(p1, on = 'List', how = 'right')
    out = out1['index']
    out[0] = 0
    out[len(out)-1] = len(end)-1
    out = list(out)
    return(out)

In [707]:
def NPV_1bd(r,k):
    
    tenor_end = Tenor_end_dates(k)
    start, end = search_buisness_days(k)
    first_day = int(pd.Series(end[0] - start[0]).dt.days)
    x = date_time_str = '01/01/1900 00:00:00'
    x = datetime.strptime(date_time_str, '%d/%m/%Y %H:%M:%S')
    
    end_int = (end[0] - x).days + 2
    start_int = (start[0] - x).days + 2
    
    dummy_zero = pd.concat([pd.DataFrame(Tenor_end_dates(k)), pd.DataFrame(Par_rates.iloc[k,1:(len(Tenors)-1)]).reset_index()], axis = 1)
    dummy_zero.iloc[0,2] = r
    
    w = (len(Tenors)-1)
    
    tenor_int = []
    
    for m in range(0,w):
        y3 = (tenor_end[m]-x).days + 2
        tenor_int.append(y3)
        
    end_zr = np.interp(end_int, tenor_int, list(dummy_zero.iloc[:,2]))
    start_zr = end_zr
    start_df = 1
    end_df = np.exp(-end_zr*first_day/365)
    forwards = fra(end_df, start_df, first_day, d_con1)
    
    fixed_leg = 25e6*first_day*Par_rates.iloc[k,1]/360
    float_leg = 25e6*first_day*forwards/360
    npv = fixed_leg - float_leg
    return(npv)

In [708]:
def newton_raphson(r,k):
    h = 0.000001
    for i in range(1,10):
        f1 = NPV_1bd(r+h,k)
        f2 = NPV_1bd(r,k)
        df = (f1-f2)/h
        r = r - f2/df
    return(r)

In [709]:
def NPV(k, zero_rates, end, start, tenor_end, r, g):
    
    x = date_time_str = '01/01/1900 00:00:00'
    x = datetime.strptime(date_time_str, '%d/%m/%Y %H:%M:%S')
    dayz = np.array(end) - np.array(start)
    dayz = pd.Series(dayz).dt.days
    
    start_int = []
    end_int = []
    tenor_int = []
    
    for m in range(len(dayz)):
        y1 = (start[m] - x).days + 2
        start_int.append(y1)
        y2 = (end[m] - x).days + 2
        end_int.append(y2)
        
    w = (len(Tenors)-1)

    for m in range(0,w):
        y3 = (tenor_end[m] - x).days + 2
        tenor_int.append(y3)
        
    dummy_zero = zero_rates
    dummy_zero.iloc[g,2] = r
    
    end_zr = np.interp(end_int, tenor_int, list(dummy_zero.iloc[:,2]))
    start_zr = np.interp(start_int, tenor_int, list(dummy_zero.iloc[:,2]))
    start_df = np.exp(-start_zr*diff_from_start(Par_rates.iloc[k,0], start, d_con2))
    end_df = np.exp(-end_zr*diff_from_start(Par_rates.iloc[k,0], end, d_con2))
    forwards = fra(end_df, start_df, dayz, d_con1)
    
    if(int(pd.Series(end[len(end)-1]-start[0]).dt.days) > 370):
        accrual_end_date = find_payment_dates(end, start, k)
        p_day = []
        payment_int = []
        
        spot = [accrual_end_date[0] + relativedelta(days = payment_lag), accrual_end_date[0] + relativedelta(days=payment_lag)]
        spot_int = (accrual_end_date[0] - x + relativedelta(days = payment_lag)).days
        
        for l in range(1, len(accrual_end_date)):
            y4 = (accrual_end_date[0] - x + relativedelta(days = payment_lag)).days + 2
            payment_int.append(y4)
            p_day.append(accrual_end_date[l])
            
        index_blocks = float_blocks(start, end, accrual_end_date)
        
        payment_zr = np.interp(payment_int, tenor_int, list(dummy_zero.iloc[:,2]))
        payment_df = np.exp(-payment_zr*diff_from_start(Par_rates.iloc[k,0], p_day, d_con2))
        
        spot_zr = np.interp(spot_int, tenor_int, list(dummy_zero.iloc[:,2]))
        spot_df = np.exp(-spot_zr*diff_from_start(Par_rates.iloc[k,0], spot, d_con2))
        
        float_leg = []
        fix_leg = []
        
        n = 0
        while(n <= len(index_blocks) - 2):
            float_leg.append(25e6*(np.prod(np.array(1 + division(dayz[int(index_blocks[n]):int(index_blocks[n+1]+1)], d_con1)*np.array(forwards[int(index_blocks[n]):int(index_blocks[n+1]+1)])) )-1))
            fix_leg.append(25e6*Par_rates.iloc[k,g+1]*division(sum(dayz[int(index_blocks[n]):int(index_blocks[n+1]+1)]), d_con1))
            n = n+1
        
        for m in range(len(float_leg)):
            float_leg[m] = float_leg[m]*payment_df[m]/spot_df[0]
            fix_leg[m] = fix_leg[m]*payment_df[m]/spot_df[0]
            
        v = sum(float_leg) - sum(fix_leg)
        return(v)
    
    else:
        
        accrual_end_date = find_payment_dates(end, start, k)
        
        spot = [accrual_end_date[0] + relativedelta(days = payment_lag), accrual_end_date[0] + relativedelta(days=payment_lag)]
        spot_int = (accrual_end_date[0] - x + relativedelta(days = payment_lag)).days
        
        spot_zr = np.interp(spot_int, tenor_int, list(dummy_zero.iloc[:,2]))
        spot_df = np.exp(-spot_zr*diff_from_start(Par_rates.iloc[k,0], spot, d_con2))
        
        float_leg = 25e6*(np.prod(np.array(1+ division(dayz, d_con1)*forwards))-1)
        fixed_leg = 25e6*Par_rates.iloc[k, g+1]*division(sum(dayz), d_con1)
        
        v = (float_leg - fixed_leg)/spot_df[0]
        return(v)
            
        


In [710]:
def newton_raphson_full(k, zero_rates, end, start, tenor_end, r, g):
    h = 0.00001
    f2 = NPV(k, zero_rates, end, start, tenor_end, r, g)
    for i in range(0,10):
        f1 = NPV(k, zero_rates,end,start,tenor_end,r+h,g)
        f2 = NPV(k, zero_rates,end,start,tenor_end,r,g)
        df = (f1 - f2)/h
        r = r - f2/df
    
    return(r)

In [713]:
k = 3
zero_rates = pd.concat([pd.DataFrame(Tenor_end_dates(k)), pd.DataFrame(Par_rates.iloc[k,1:(len(Tenors))]).reset_index()], axis = 1)
start, end = search_buisness_days(k)
tenor_end = Tenor_end_dates(k)

for t in range(0, len(Tenors)-1):
    if(t == 0):
        zero_rates.iloc[t,2] = newton_raphson(0.004,k)

    else:
        zero_rates.iloc[t,2] = newton_raphson_full(k, zero_rates, end[t], start[t], tenor_end, Par_rates.iloc[k,t], t)
            

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [714]:
zero_rates

,0,index,3
0,2023-08-09,ON,0.050684
1,2023-08-15,1W,0.050669
2,2023-08-22,2W,0.050644
3,2023-09-08,1M,0.050585
4,2023-10-08,2M,0.050481
5,2023-11-08,3M,0.050373
6,2023-12-08,4M,0.05027
7,2024-01-08,5M,0.050163
8,2024-02-08,6M,0.050057
9,2024-03-08,7M,0.049959
